In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from bs4 import BeautifulSoup as bs

In [5]:
html = "<b>Items</b><ul><li>x</li><li><b>y</b></li><li>z</li></ul>"
doc = bs(html, "html.parser")
doc

<b>Items</b><ul><li>x</li><li><b>y</b></li><li>z</li></ul>

In [8]:
print(doc.prettify())

<b>
 Items
</b>
<ul>
 <li>
  x
 </li>
 <li>
  <b>
   y
  </b>
 </li>
 <li>
  z
 </li>
</ul>


In [11]:
elements = doc.find_all('li')
print(len(elements))

3


In [12]:
for e in elements:
    print(e.get_text())

x
y
z


In [13]:
for i in doc:
    print(i.get_text())

Items
xyz


In [15]:
elements = doc.find_all("b")
print(len(elements))

2


In [18]:
li = doc.find('li')
assert(li != None)
li

<li>x</li>

In [19]:
bold = doc.find('ul').find_all("b")

In [20]:
print(bold)

[<b>y</b>]


In [21]:
#elements may contain atributes, text, other elements

In [23]:
<a href="schedule.html"><i>please</i>click<b>here</b></a>

SyntaxError: invalid syntax (<ipython-input-23-8a4d948eafb6>, line 1)

In [24]:
#demo

In [30]:
import requests as r
import os

In [31]:
url = "https://simple.wikipedia.org/wiki/List_of_U.S._states"
res = r.get(url)
res.raise_for_status()
res.text #huge string

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of U.S. states - Simple English Wikipedia, the free encyclopedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpJ9KQpAICMAAGYNoHEAAAEO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_U.S._states","wgTitle":"List of U.S. states","wgCurRevisionId":6874248,"wgRevisionId":6874248,"wgArticleId":3023,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["States of the United States","Lists about U.S. states"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRele

In [33]:
soup = bs(res.text,"html.parser")

In [34]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of U.S. states - Simple English Wikipedia, the free encyclopedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpJ9KQpAICMAAGYNoHEAAAEO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_U.S._states","wgTitle":"List of U.S. states","wgCurRevisionId":6874248,"wgRevisionId":6874248,"wgArticleId":3023,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["States of the United States","Lists about U.S. states"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitex

In [37]:
tables = soup.find_all("table")#only one table
print(len(tables))
assert len(tables)==1
tbl = tables[0]

1


In [39]:
trs = tbl.find_all("tr")
assert len(trs)>=50

In [54]:
prefix = "https://simple.wikipedia.org"
state_links = {}
for tr in trs:
    th = tr.find('th')
    links = th.find_all("a")
    if len(links)==0:
        continue
    link = links[0]
    state = link.get_text()
    state_url = prefix + link.attrs["href"]
    state_links[state] = state_url
state_links

{'postal abbreviation': 'https://simple.wikipedia.org/wiki/List_of_U.S._state_abbreviations',
 'Alabama': 'https://simple.wikipedia.org/wiki/Alabama',
 'Alaska': 'https://simple.wikipedia.org/wiki/Alaska',
 'Arizona': 'https://simple.wikipedia.org/wiki/Arizona',
 'Arkansas': 'https://simple.wikipedia.org/wiki/Arkansas',
 'California': 'https://simple.wikipedia.org/wiki/California',
 'Colorado': 'https://simple.wikipedia.org/wiki/Colorado',
 'Connecticut': 'https://simple.wikipedia.org/wiki/Connecticut',
 'Delaware': 'https://simple.wikipedia.org/wiki/Delaware',
 'Florida': 'https://simple.wikipedia.org/wiki/Florida',
 'Georgia': 'https://simple.wikipedia.org/wiki/Georgia_(U.S._state)',
 'Hawaii': 'https://simple.wikipedia.org/wiki/Hawaii',
 'Idaho': 'https://simple.wikipedia.org/wiki/Idaho',
 'Illinois': 'https://simple.wikipedia.org/wiki/Illinois',
 'Indiana': 'https://simple.wikipedia.org/wiki/Indiana',
 'Iowa': 'https://simple.wikipedia.org/wiki/Iowa',
 'Kansas': 'https://simple.wik

In [74]:
#stage 2
html_dir = "html_files_for_states"
state_files={}
for state in state_links.keys():
    if state == "postal abbreviation":
        continue
    state_url = state_links[state]
    res = r.get(state_url)
    state_file = os.path.join(html_dir, state + ".html")
    state_files[state]= state_file
    if os.path.exists(state_file):
        continue
    f = open(state_file, "w", encoding= "utf-8")
    f.write(res.text)
    f.close()

In [101]:
#stage 3
def state_stats(path):
    stats = {}
    f = open(path, "r",encoding = "utf-8")
    html_string = f.read()
    f.close()
    doc = bs(html_string, "html.parser")
    trs = doc.find_all("tr")
    for tr in trs:
        cells = tr.find_all(["th","td"])
        if len(cells) == 2:
            key = cells[0].get_text()
            val = cells[1].get_text()
            stats[key] = val
    return stats

In [102]:
state_stats(state_files['Wisconsin']).keys()

dict_keys(['Country', 'Before statehood', 'Admitted to the Union', 'Largest city', 'Largest metro', '\xa0•\xa0Governor', '\xa0•\xa0Lieutenant Governor', 'Legislature', '\xa0•\xa0Upper house', '\xa0•\xa0Lower house', 'U.S. senators', 'U.S. House delegation', '\xa0•\xa0Total', '\xa0•\xa0Density', '\xa0•\xa0Median household income', '\xa0•\xa0Income rank', 'Trad. abbreviation', 'Latitude', 'Longitude', 'Bird', 'Fish', 'Flower', 'Insect', 'Tree', 'Beverage', 'Dance', 'Food', 'Fossil', 'Mineral', 'Rock', 'Slogan', 'Soil', 'Tartan', '', 'States', 'Federal district', 'Insular areas', 'Outlying islands', 'East North Central', 'West North Central'])

In [103]:
states_dets = []
for state in state_files.keys():
    stats = state_stats(state_files[state])
    states_dets.append(stats)

In [105]:
import pandas as pd

In [106]:
states_df = pd.DataFrame(states_dets)

In [107]:
states_df

Country                                   Before statehood  \
0   United States                                  Alabama Territory   
1   United States                                Territory of Alaska   
2   United States                                  Arizona Territory   
3   United States                                 Arkansas Territory   
4   United States                                    Alta California   
5   United States                                                NaN   
6   United States                                 Connecticut Colony   
7   United States        Delaware Colony, New Netherland, New Sweden   
8   United States                                  Florida Territory   
9   United States                                Province of Georgia   
10  United States                                Territory of Hawaii   
11  United States  Oregon Territory, Washington Territory, Idaho ...   
12  United States                                 Illinois Territory   
13  United States                                  Indiana Territory   
14  United States                                     Iowa Territory   
15  United States                                   Kansas Territory   
16  United States            Part of Virginia (District of Kentucky)   
17  United States                               Territory of Orleans   
18  United States                  District of Maine (Massachusetts)   
19  United States                               Province of Maryland   
20  United States                      Province of Massachusetts Bay   
21  United States                                 Michigan Territory   
22  United States                                Minnesota Territory   
23  United States                              Mississippi Territory   
24  United States                                 Missouri Territory   
25  United States                                  Montana Territory   
26  United States                                 Nebraska Territory   
27  United States  Nevada Territory, Utah Territory, New Mexico T...   
28  United States                          Province of New Hampshire   
29  United States                             Province of New Jersey   
30  United States  Nuevo México (1598-1848)New Mexico Territory (...   
31  United States                                                NaN   
32  United States                         Province of North-Carolina   
33  United States                                   Dakota Territory   
34  United States                                                NaN   
35  United States  Indian Territory (1834–1907)Oklahoma Territory...   
36  United States                                   Oregon Territory   
37  United States                           Province of Pennsylvania   
38  United States  Colony of Rhode Island and Providence Plantations   
39  United States                         Province of South Carolina   
40  United States                                   Dakota Territory   
41  United States                                Southwest Territory   
42  United States                                  Republic of Texas   
43  United States                                     Utah Territory   
44  United States                                   Vermont Republic   
45  United States                                 Colony of Virginia   
46  United States                               Washington Territory   
47  United States                                   Part of Virginia   
48  United States                                Wisconsin Territory   
49  United States                                  Wyoming Territory   

                                Admitted to the Union          Largest city  \
0                            December 14, 1819 (22nd)            Birmingham   
1                              January 3, 1959 (49th)             Anchorage   
2                            February 14, 1912 (48th)                   NaN   
3                    